<a href="https://colab.research.google.com/github/Sukhwinder9813/EmojiPredictionfromTweet/blob/master/EmojiPredictorfromTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


In [2]:
import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\zhangkerou\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\zhangkerou\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\zhangkerou\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\zhangkerou\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\zhangkerou\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     C:\Users

True

In [6]:
import emoji

In [7]:
from keras.preprocessing.text import Tokenizer

In [8]:
from nltk.corpus import stopwords


In [10]:
data=pd.read_csv('Train.csv')

In [11]:
x_train=data['TEXT'].values

In [12]:
y_train=data['Label'].values

In [13]:
smoothtweets=[]
stopper=set(stopwords.words('english'))
for tweets in x_train:
    words=tweets.split(" ")
    str = ""
    for word in words:
        if word[0] != "@" and word not in stopper:
            if word[0] == "#":
                word = word[1:]
        str += word + " "
    smoothtweets.append(str)

In [14]:
smoothtweets[2]

'Been friends since 7th grade. Look at us now we all following our dreams doing what we love and…\n '

In [15]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', split=" ", lower=True)
tokenizer.fit_on_texts(smoothtweets)
newsmoothtweets=[]
newsmoothtweets=tokenizer.texts_to_sequences(smoothtweets)

In [16]:
print(newsmoothtweets[3])

[14, 16, 65, 24, 430, 53, 44, 487, 375, 10, 10236, 213, 1047, 519, 10237]


In [17]:
from keras.preprocessing import sequence
X_train=sequence.pad_sequences(newsmoothtweets,maxlen=12,padding='post')

In [18]:
from keras.layers import *
from keras.models import Sequential 

In [19]:
from keras.utils import to_categorical

In [20]:
Y_train=to_categorical(y_train)

In [21]:
Y_train.shape

(70000, 20)

In [22]:
X_train.shape

(70000, 12)

In [27]:
wordembedd={}
f=open('glove.6B.50d.txt',"r",encoding='utf-8')
for line in f:
  words=line.split()
  word=words[0]
  coefs=np.asarray(words[1:],dtype='float')
  wordembedd[word]=coefs
f.close()

In [28]:
def populate_weight_matrix(vocab, raw_embedding):
    # Create weight matrix from pre-trained embeddings
    vocab_size = len(vocab) + 1
    weight_matrix = np.zeros((vocab_size, 50))
    for word, i in vocab.items():
        if word in raw_embedding:
            weight_matrix[i] = raw_embedding[word]
    return weight_matrix

In [29]:
vocab=tokenizer.word_index

In [30]:
len(vocab)
import math

In [31]:
weight_matrix=populate_weight_matrix(vocab,wordembedd)

In [32]:
weight_matrix[8]

array([ 0.26818 ,  0.14346 , -0.27877 ,  0.016257,  0.11384 ,  0.69923 ,
       -0.51332 , -0.47368 , -0.33075 , -0.13834 ,  0.2702  ,  0.30938 ,
       -0.45012 , -0.4127  , -0.09932 ,  0.038085,  0.029749,  0.10076 ,
       -0.25058 , -0.51818 ,  0.34558 ,  0.44922 ,  0.48791 , -0.080866,
       -0.10121 , -1.3777  , -0.10866 , -0.23201 ,  0.012839, -0.46508 ,
        3.8463  ,  0.31362 ,  0.13643 , -0.52244 ,  0.3302  ,  0.33707 ,
       -0.35601 ,  0.32431 ,  0.12041 ,  0.3512  , -0.069043,  0.36885 ,
        0.25168 , -0.24517 ,  0.25381 ,  0.1367  , -0.31178 , -0.6321  ,
       -0.25028 , -0.38097 ])

In [33]:
max_length = math.ceil(sum([len(s.split(" ")) for s in smoothtweets])/len(smoothtweets))
vocab_size=len(vocab)+1
model=Sequential()
model.add(Embedding(vocab_size,50,weights=[weight_matrix],input_length=max_length+2,trainable=True,))
model.add(LSTM(128,dropout=0.2,return_sequences=True))
model.add(LSTM(128,dropout=0.2))
model.add(Dense(20,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 15, 50)            4276550   
_________________________________________________________________
lstm (LSTM)                  (None, 15, 128)           91648     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 20)                2580      
Total params: 4,502,362
Trainable params: 4,502,362
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.fit(X_train,Y_train,epochs=5,batch_size=128,shuffle=True,validation_split=0.15)

Epoch 1/5
465/465 [==============================] - 22s 48ms/step - loss: 2.5053 - accuracy: 0.2649 - val_loss: 2.2952 - val_accuracy: 0.3121
Epoch 2/5
465/465 [==============================] - 21s 46ms/step - loss: 2.2421 - accuracy: 0.3274 - val_loss: 2.2234 - val_accuracy: 0.3305
Epoch 3/5
465/465 [==============================] - 22s 47ms/step - loss: 2.1137 - accuracy: 0.3621 - val_loss: 2.2000 - val_accuracy: 0.3353
Epoch 4/5
465/465 [==============================] - 22s 47ms/step - loss: 1.9647 - accuracy: 0.4043 - val_loss: 2.2169 - val_accuracy: 0.3404
Epoch 5/5
465/465 [==============================] - 22s 47ms/step - loss: 1.7812 - accuracy: 0.4557 - val_loss: 2.2871 - val_accuracy: 0.3321


In [36]:
model.evaluate(X_train,Y_train)

2188/2188 [==============================] - 7s 3ms/step - loss: 1.6001 - accuracy: 0.5195


[1.600089430809021, 0.5194857120513916]

In [37]:
model.predict_classes(X_train[1:13])

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([ 1, 13,  3,  3,  7,  5,  1,  2, 16,  9,  9,  4], dtype=int64)

In [38]:
smoothtweets[9]

'Story On Saturday 136 Collins Ave, Miami Beach, FL 33139 Ladies Contact Me To Join Me In my…\n '

In [39]:
df=pd.read_csv('Mapping.csv')

In [40]:
df.head(20)

,Unnamed: 0,emoticons,number
0,0,😜,0
1,1,📸,1
2,2,😍,2
3,3,😂,3
4,4,😉,4
5,5,🎄,5
6,6,📷,6
7,7,🔥,7
8,8,😘,8
9,9,❤,9


In [41]:
data['Label'][9]

7

In [42]:
df3=pd.read_csv('Test.csv')

In [43]:
df3.head()

,Unnamed: 0,id,TEXT
0,0,0,Thought this was cool...#Repost (get_repost)・・...
1,1,1,Happy 4th! Corte madera parade. #everytownusa ...
2,2,2,Luv. Or at least something close to it. @ Unio...
3,3,3,There's a slice of pie under that whipped crea...
4,5,5,#thankyou for your thank you We adore you both...


In [44]:
xtest=df3['TEXT']


In [45]:
smoothTESTtweets=[]
stopper=set(stopwords.words('english'))
for tweets in xtest:
  words=tweets.split(" ")
  str = ""
  for word in words:
      if word[0] != "@" and word not in stopper:
          if word[0] == "#":
              word = word[1:]
          str += word + " "
  smoothTESTtweets.append(str)

In [46]:
newsmoothtweets=tokenizer.texts_to_sequences(smoothTESTtweets)

In [47]:
newsmoothtweets[2]

[1240, 163, 1132, 462, 842, 24, 776, 523, 1621, 351]

In [48]:
x_test=sequence.pad_sequences(newsmoothtweets,maxlen=12,padding='post')

In [49]:
x_test[3]

array([ 716, 7386, 1682, 5295,  770,  126, 6904,    0,    0,    0,    0,
          0])

In [50]:
data=model.predict_classes(x_test)


In [51]:
for i in range(20,35,1):
  print(df['emoticons'][data[i]])
  print(smoothTESTtweets[i])

😂
One favorite gyms FREE Fucking membership, get ass . Balboa Park
 
😊
Hanging guy today Thank great shoot filled plenty of…
 
😎
mood today Parkway North High School
 
😍
Chandelier Widener Library
 
😂
Sorry jump bandwagon. thisfilteriseverything Las…
 
❤
️ Princeton, New Jersey
 
❤
My main man Gladstone, Missouri
 
😂
Eat shit Irma ghostriotradio hurricaneirma irmanators goodforsprinkle pbr…
 
❤
Max went first aquarium trip went moving walkway. He amazing! ️ Check out…
 
😍
ikea fun me ... oh dam store, like disney interior architect's IKEA
 
😂
It's like I blinked 2 years flew Short Pump Town Center
 
💙
When moment meet future husband!!! marlonovershawn teammarlon Stress…
 
😎
We working hard today McDonald's 16827 Marsh Rd
 
🔥
"No Flocking" shot coming soon! LSV2 available datpiff freestyle…
 
🔥
Dj Ivan G! One Mixer, One Man, One Dj, That Rocks The Party! Always Mixes for…
 


In [52]:
smoothTESTtweets[2]

'Luv. Or least something close it. Union Hill, Richmond, Virginia\n '

In [55]:
xtest = ["I love you"]

smoothTESTtweets=[]
stopper=set(stopwords.words('english'))
for tweets in xtest:
  words=tweets.split(" ")
  str = ""
  for word in words:
      if word[0] != "@" and word not in stopper:
          if word[0] == "#":
              word = word[1:]
          str += word + " "
  smoothTESTtweets.append(str)

newsmoothtweets=tokenizer.texts_to_sequences(smoothTESTtweets)

x_test=sequence.pad_sequences(newsmoothtweets,maxlen=12,padding='post')
data=model.predict_classes(x_test)

In [57]:
print(df['emoticons'][data[0]])
print(smoothTESTtweets[0])

❤
I love 
